In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.impute import KNNImputer
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def merge(df, cols, name, inverse=False):

  if inverse == True:
    if len(cols) != 2:
      return -1

    a = df[cols[0]]
    b = np.invert(df[cols[1]])
    df[name] = np.logical_or(a, b)

  else:
    
    df[name] = df[cols[0]]
  #pd.Series([np.nan for x in range(len(df.index))], index=df.index)

    for col in df[cols].columns:
      df[name] = np.logical_or(df[name], df[col])

    df[name] = df[name]*1


In [ ]:
raw = pd.read_csv("drive/My Drive/COSIW/clean_6.csv")

In [ ]:
raw.head()

,V00995,hospitalized_depression,anti_depress_12mnth,tranquil_12mnth,amphetanines_12mnth,anti_psychotic_12mnth,ins_military,ins_obtained_employer,ins_purchased_company,ins_medicare,ins_government_prog,ins_state_health,ins_other,recieved_welfare,child_died,non_english_home_lang,lived_biolog_parents,depression_tendency,depress_diagnos,children_HH_0,children_HH_1,children_HH_2,children_HH_3,children_HH_4,adults_HH_0,adults_HH_1,adults_HH_2,adults_HH_3,adults_HH_4,adults_HH_5,identify_ethnicity_1_1.0,identify_ethnicity_1_2.0,identify_ethnicity_1_3.0,identify_ethnicity_1_4.0,identify_ethnicity_2_1.0,identify_ethnicity_2_2.0,identify_ethnicity_2_3.0,identify_ethnicity_2_4.0,cost_routine_checkup_0,cost_routine_checkup_1-5,...,imp_religion_3.0,imp_religion_4.0,neighborhood_1.0,neighborhood_2.0,neighborhood_3.0,neighborhood_4.0,neighborhood_5.0,neighborhood_6.0,religion_1.0,religion_2.0,religion_3.0,religion_4.0,religion_5.0,religion_6.0,religion_7.0,religion_8.0,religion_9.0,religion_10.0,religion_11.0,religion_12.0,religion_13.0,religion_14.0,identify_ethnicity_1.0,identify_ethnicity_1.5,identify_ethnicity_2.0,identify_ethnicity_2.5,identify_ethnicity_3.0,identify_ethnicity_3.5,identify_ethnicity_4.0,prob_comf_talk_prof_1.0,prob_comf_talk_prof_1.5,prob_comf_talk_prof_2.0,prob_comf_talk_prof_2.5,prob_comf_talk_prof_3.0,prob_comf_talk_prof_3.5,prob_comf_talk_prof_4.0,sex_1,sex_2,primary_secondary_adult_1.0,primary_secondary_adult_2.0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,1,0
1,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,1,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1
3,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0
4,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0


In [ ]:
raw.shape

(4000, 147)

In [ ]:
new_data = raw.copy(deep=True)

In [ ]:
imputer = KNNImputer(n_neighbors=5)

In [ ]:
imputed_new_data = imputer.fit_transform(new_data)

In [ ]:
np.isnan(imputed_new_data).sum()

0

In [ ]:
imputed_new_data_df = pd.DataFrame(data=imputed_new_data, columns=new_data.columns)

In [ ]:
threshold_data = imputed_new_data_df.copy(deep=True)

In [ ]:
for col in threshold_data.columns:
  if len(threshold_data[col].unique()) > 2:
    threshold_data[col] = threshold_data[col].apply(lambda x: 1 if x >= 0.5 else 0)

In [ ]:
threshold_data.to_csv("drive/My Drive/COSIW/imputed_2.csv", index=False)

# NEW IMPUTATION

In [ ]:
raw = pd.read_csv("drive/My Drive/COSIW/a_5.csv")

In [ ]:
raw.head()

,RANCEST,prob_talk_prof,comf_talk_prof,embar_talk_prof,%_helpful_talk_prof,%_better_without_prof,religion_comfort,inter_hard_english,treated_badly_english,avoid_treatment_INS,comm_care_own_lang,interp_available,cost_routine_checkup,cost_perscrip_medicine,difficult_appointment_phone,difficult_referral_specialist,provider_spends_limited_time,1_hr_waiting_room,difficult_advice_phone,difficult_going_clinic,difficult_getting_meds,lack_continuous_care_providers,V00995
0,hispanic,1.0,1.0,4.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5-10,10-16,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,hispanic,3.0,2.0,4.0,NaN,NaN,4.0,0.0,0.0,0.0,NaN,NaN,16-20,20-50,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,hispanic,1.0,1.0,4.0,NaN,NaN,1.0,0.0,1.0,0.0,1.0,NaN,10-16,50-750,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
3,hispanic,1.0,1.0,4.0,NaN,NaN,1.0,1.0,1.0,0.0,1.0,NaN,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,asian,3.0,2.0,3.0,NaN,NaN,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
raw.shape

(3476, 23)

In [ ]:
raw.isna().sum() < 1500


RANCEST                            True
prob_talk_prof                     True
comf_talk_prof                     True
embar_talk_prof                    True
%_helpful_talk_prof               False
%_better_without_prof             False
religion_comfort                   True
inter_hard_english                False
treated_badly_english             False
avoid_treatment_INS               False
comm_care_own_lang                False
interp_available                  False
cost_routine_checkup               True
cost_perscrip_medicine            False
difficult_appointment_phone       False
difficult_referral_specialist     False
provider_spends_limited_time      False
1_hr_waiting_room                 False
difficult_advice_phone            False
difficult_going_clinic            False
difficult_getting_meds            False
lack_continuous_care_providers    False
V00995                             True
dtype: bool

In [ ]:
data = raw.copy(deep=True)

In [ ]:
cited_difficulty_cols = ["difficult_appointment_phone", "difficult_referral_specialist", "provider_spends_limited_time", "1_hr_waiting_room", "difficult_advice_phone", "difficult_going_clinic", "difficult_getting_meds", "lack_continuous_care_providers"]
merge(data, cited_difficulty_cols, "cited_difficulty")

In [ ]:
data.isna().sum()

RANCEST                              0
prob_talk_prof                     633
comf_talk_prof                     643
embar_talk_prof                    636
%_helpful_talk_prof               1611
%_better_without_prof             1607
religion_comfort                   810
inter_hard_english                3011
treated_badly_english             3012
avoid_treatment_INS               3011
comm_care_own_lang                3132
interp_available                  3457
cost_routine_checkup              1331
cost_perscrip_medicine            1530
difficult_appointment_phone       2877
difficult_referral_specialist     2877
provider_spends_limited_time      2878
1_hr_waiting_room                 2877
difficult_advice_phone            2879
difficult_going_clinic            2878
difficult_getting_meds            2877
lack_continuous_care_providers    2877
V00995                               0
cited_difficulty                     0
dtype: int64

In [ ]:
data.drop(columns=cited_difficulty_cols, inplace=True)

In [ ]:
data.isna().sum()

RANCEST                      0
prob_talk_prof             633
comf_talk_prof             643
embar_talk_prof            636
%_helpful_talk_prof       1611
%_better_without_prof     1607
religion_comfort           810
inter_hard_english        3011
treated_badly_english     3012
avoid_treatment_INS       3011
comm_care_own_lang        3132
interp_available          3457
cost_routine_checkup      1331
cost_perscrip_medicine    1530
V00995                       0
cited_difficulty             0
dtype: int64

In [ ]:
drop_these = [col for col in data.columns if data[col].isna().sum() > 3000]

In [ ]:
data.drop(columns=drop_these, inplace=True)

In [ ]:
data.isna().sum()

RANCEST                      0
prob_talk_prof             633
comf_talk_prof             643
embar_talk_prof            636
%_helpful_talk_prof       1611
%_better_without_prof     1607
religion_comfort           810
cost_routine_checkup      1331
cost_perscrip_medicine    1530
V00995                       0
cited_difficulty             0
dtype: int64

In [ ]:
for col in data.columns:
  print(col + " " + str(data[col].unique()))

RANCEST ['hispanic' 'asian' 'white' 'black' 'other']
prob_talk_prof [ 1.  3.  2.  4. nan]
comf_talk_prof [ 1.  2.  4.  3. nan]
embar_talk_prof [ 4.  3.  2.  1. nan]
%_helpful_talk_prof [nan '25-50' '75-100' '0-25' '50-75']
%_better_without_prof [nan '0-25' '25-50' '50-75' '75-100']
religion_comfort [ 1.  4.  3.  2. nan]
cost_routine_checkup ['5-10' '16-20' '10-16' '0' nan '1-5' '20-50' '50-750']
cost_perscrip_medicine ['10-16' '20-50' '50-750' '0' nan '5-10' '1-5' '16-20']
V00995 [1. 0.]
cited_difficulty [1 0]


In [ ]:
one_hot_cols = []

for col in data.columns:
  if data[col].nunique() > 2:
    one_hot_cols.append(col)

one_hot_df = pd.get_dummies(data, columns=one_hot_cols)

In [ ]:
for col in one_hot_cols:
  one_hot_df.loc[data[col].isnull(), one_hot_df.columns.str.startswith(col)] = np.nan

In [ ]:
imputer = KNNImputer(n_neighbors=5)

In [ ]:
imputed_new_data = imputer.fit_transform(one_hot_df)

In [ ]:
imputed_new_data_df = pd.DataFrame(data=imputed_new_data, columns=one_hot_df.columns)

In [ ]:
imputed_new_data_df.head()

,V00995,cited_difficulty,RANCEST_asian,RANCEST_black,RANCEST_hispanic,RANCEST_other,RANCEST_white,prob_talk_prof_1.0,prob_talk_prof_2.0,prob_talk_prof_3.0,prob_talk_prof_4.0,comf_talk_prof_1.0,comf_talk_prof_2.0,comf_talk_prof_3.0,comf_talk_prof_4.0,embar_talk_prof_1.0,embar_talk_prof_2.0,embar_talk_prof_3.0,embar_talk_prof_4.0,%_helpful_talk_prof_0-25,%_helpful_talk_prof_25-50,%_helpful_talk_prof_50-75,%_helpful_talk_prof_75-100,%_better_without_prof_0-25,%_better_without_prof_25-50,%_better_without_prof_50-75,%_better_without_prof_75-100,religion_comfort_1.0,religion_comfort_2.0,religion_comfort_3.0,religion_comfort_4.0,cost_routine_checkup_0,cost_routine_checkup_1-5,cost_routine_checkup_10-16,cost_routine_checkup_16-20,cost_routine_checkup_20-50,cost_routine_checkup_5-10,cost_routine_checkup_50-750,cost_perscrip_medicine_0,cost_perscrip_medicine_1-5,cost_perscrip_medicine_10-16,cost_perscrip_medicine_16-20,cost_perscrip_medicine_20-50,cost_perscrip_medicine_5-10,cost_perscrip_medicine_50-750
0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.4,0.0,0.6,0.8,0.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.4,0.2,0.2,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.2,0.0,0.8,0.8,0.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.2,0.2,0.6,0.6,0.4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.6,0.4,0.0,0.2,0.6,0.0,0.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.2,0.2,0.4,0.2,0.2,0.0,0.0,0.2,0.0,0.6,0.0


In [ ]:
threshold_data = imputed_new_data_df.copy(deep=True)

In [ ]:
for col in threshold_data.columns:
  if len(threshold_data[col].unique()) > 2:
    threshold_data[col] = threshold_data[col].apply(lambda x: 1 if x >= 0.5 else 0)

In [ ]:
threshold_data.to_csv("drive/My Drive/COSIW/a_imputed.csv", index=False)